In [102]:
import os
import json
import requests
import pandas as pd

In [103]:
autority = pd.read_parquet(os.path.join("data","aut_vyber.parquet"))

In [104]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [105]:
autority = autority[autority['024_2'].notnull()]
autority = autority[autority['024_a'].notnull()]

In [106]:
autority.sample(20)

,024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,370_b,370_c,370_f,372_a,373_a,374_a,375_a,377_a,400_a,400_d,400_i,400_ind1,410_a,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader
001,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ntk20201090779,"[isni, wikidata, orcid]","[0000000038701137, Q37836657, 0000-0001-8540-6...",[1959],None,[ntk20201090779],"[Bagetta, Giacinto,]",[1959-],[1],None,None,None,[Itálie],None,"[lékařství, farmakologie, neurovědy, neurodege...",[Università degli Studi della Calabria],"[farmakologové, biologové, vysokoškolští učite...",[muž],"[eng, ita]",None,None,None,None,None,None,None,None,None,None,None,None,"[Italský biolog, doktor medicíny, farmakolog, ...",None,nz a22 n 4500
mub20231172516,"[isni, wikidata]","[0000000425209572, Q20034006]",[1908],[2000],[mub20231172516],"[Mel‘nikov, Nikolaj Nikolajevič,]",[1908-2000],[1],None,"[Moskva, Rusko]","[Moskva, Rusko]",[Rusko],[Sovětský svaz],"[organická chemie, pesticidy, chemická ochrana...",None,"[chemici, vysokoškolští učitelé, redaktoři]",[muž],"[rus, eng]","[Mel‘nikov, N. N., Mel‘nikov, N. N.,, Мельнико...","[1908-2000, 1908-2000, 1908-2000]",None,"[1, 1, 1]",None,None,None,None,None,None,None,None,"[Ruský a sovětský chemik, specialista v oboru ...",None,cz a22 n 4500
mzk20221171797,"[isni, wikidata]","[0000000408731190, Q95315057]",[1983],None,[mzk20221171797],"[Benz, Maximilian,]",[1983-],[1],None,"[Mnichov, Německo]",None,"[Německo, Švýcarsko]",None,"[literární věda, germanistika, německá literat...",None,"[literární vědci, germanisté, vysokoškolští uč...",[muž],[ger],"[Benz, Maximilian Johannes,]",[1983-],None,[1],None,None,None,None,None,None,None,None,"[Německý literární vědec, germanista a vysokoš...",None,nz a22 n 4500
mzk20221157381,[wikidata],[Q113360501],[1993],None,[mzk20221157381],"[Netolická, Karolína,]",[1993-],[1],None,"[Praha, Česko]",None,[Česko],None,[malířství],None,[malířky],[žena],[cze],None,None,None,None,None,None,None,None,None,None,None,None,[Narozena 21. 8. 1993 v Praze. Malířka.],None,nz a22 n 4500
nlk20000085709,"[wikidata, isni]","[Q18674317, 0000000058502266]",[1851],[1921],[nlk20000085709],"[Freyer, P. J.]",[1851-1921],[1],None,None,None,"[Irsko, Velká Británie]",None,None,None,"[lékaři, chirurgové, vysokoškolští učitelé]",[muž],[cze],"[Freyer, Peter Johnson,, Freyer, Peter Johnsto...","[1851-1921, 1851-1921, 1851-1921]",None,"[1, 1, 1]",None,None,None,None,None,None,None,None,[Lékař.],None,cz a22 n 4500
ntk20221154294,"[isni, orcid]","[0000000116444464, 0000-0002-7706-1915]",[1952],None,[ntk20221154294],"[Gullino, Maria Lodovica,]",[1952-],[1],None,None,None,[Itálie],None,"[fytopatologie, nemoci rostlin, biologická bez...",[Università di Torino],"[vysokoškolské učitelky, vědkyně]",[žena],"[ita, eng]","[Gullino, M. Lodovica, Gullino, M. Lodovica,, ...","[1952-, 1952-, 1952-, 1952-]",None,"[1, 1, 1, 1]",None,None,None,None,None,None,None,None,[Italská vysokoškolská pedagožka a výzkumná pr...,None,cz a22 n 4500
ctu2017948885,[orcid],[0000-0001-9224-2151],[1983],None,[ctu2017948885],"[Vonásek, Vojtěch,]",[1983-],[1],None,"[Pelhřimov, Česko]",None,[Česko],None,"[umělá inteligence, biokybernetika]",None,"[vysokoškolští učitelé, elektroinženýři]",[muž],"[cze, eng]",None,None,None,None,None,None,None,None,None,None,None,None,[Narozen v roce 1983 v Pelhřimově. Elektroinže...,None,cz a22 n 4500
jo20231198722,"[isni, wikidata]","[0000000114267273, Q2997661]",[1882],[1941],[jo20231198722],"[Cavid, Hüseyn,]",[1882-1941],[1],None,None,None,[Ázerbájdžán],None,[romantismus (literatura)],None,"[básníci, spisovatelé, dramatici]",[muž],[aze],"[Cavid, Hüseyn, Rasizadä, Hüseyn Abdulla oğlu,...","[1882-1941, 1882-1941, 1882-1941, 1882-1941, 1...",None,"[1, 1, 1, 1, 1, 1]",None,None,None,None,None,None,None,None,"[Ázerbájdžánský básník, spisovatel a dramatik.]",None,nz a22 n 4500
ola20221164753,"[wikidata, isni]","[Q106822867, 000000050468251X]",None,None,[ola20221164753],"[Hardorf, Alexandra]",None,[1],None,None,None,[Německo]

In [107]:
autority = autority.explode(['024_a','024_2'])

In [108]:
wikids = autority[autority['024_2'] == 'wikidata']['024_a'].drop_duplicates().to_list()

In [127]:
aut_wikids = autority[autority['024_2'] == 'wikidata']['024_a'].drop_duplicates(keep="first")
len(aut_wikids)

61384

In [129]:
aut_wikids = aut_wikids[~aut_wikids.index.duplicated(keep='first')]
len(aut_wikids)

61354

In [151]:
pd.DataFrame(aut_wikids).to_json(os.path.join("data_raw","autority_wikidataids.json"))

In [153]:
pd.DataFrame(aut_wikids)

,024_a
001,
jk01010005,Q1063371
jk01010022,Q12028104
jk01010023,Q640512
jk01010026,Q12055885
jk01010030,Q634048
...,...
xx0323358,Q93240837
xx0323360,Q77014514
xx0323362,Q95199390


In [112]:
len(wikids)

61384

In [145]:
wikids[0:5]

['Q1063371', 'Q12028104', 'Q640512', 'Q12055885', 'Q634048']

In [113]:
kam = 'downloads/wikidata/autority'

In [114]:
if not os.path.exists(kam):
    os.makedirs(kam)

In [133]:
vsechny_jsony = [f for f in os.listdir(kam)]

In [135]:
len(vsechny_jsony)

21535

In [117]:
def get_wikidata_json(entity_id):
    
    url = f"https://www.wikidata.org/wiki/Special:EntityData/{entity_id}.json"
    response = requests.get(url)

    if response.status_code == 200:
        with open(os.path.join(kam, f"{entity_id}.json"), "w+", encoding="utf-8") as file:
            json.dump(response.json(), file, ensure_ascii=False, indent=4)
        print(f"Data for {entity_id} has been saved to {entity_id}.json")
    else:
        print(f"Failed to retrieve data: {response.status_code}")

In [143]:
for w in wikids:
    if f"{w}.json" not in vsechny_jsony:
        get_wikidata_json(w)

Failed to retrieve data: 400
Failed to retrieve data: 400



KeyboardInterrupt

